# Create data for Train/Test/Val

### This data is, though synthetic, also real and does not have the same drawbacks as general synthetic data

### Training Amounts:
1 `.cnf` file amounts to 1 piece of training data. So, there will be the following number of datapoints for the respective categories:

* Training: 5 million files
* Validation: 250 thousand files
* Testing: 100 thousand files

## Testing

In [ ]:
import sys 
import os
sys.path.append(os.path.abspath('..'))

from utils import sts3sat as s3s
from utils import sat_scrambler as ss

ORDER = 25
NUM_FILES = 100000
DATA_TYPE = 'test'
DIR_PATH = '/work/luduslab/sts_3sat/sols_by_confs/cnf_data/'

for i in range(NUM_FILES):
    file_path = DIR_PATH + f'{DATA_TYPE}/{DATA_TYPE}_no{i}.cnf'

    sts = s3s.generateSteinerTripleSystem(v=ORDER)
    s3s.sts_to_sat_file(sts=sts, order=ORDER, file_path=file_path)
    seed = ss.generate_random_seed(order=ORDER)
    ss.packaged_scrambler(base_file_path=file_path, seed=seed, write_file_path=file_path)


## Validation

In [ ]:
import sys 
import os
sys.path.append(os.path.abspath('..'))

from utils import sts3sat as s3s
from utils import sat_scrambler as ss

ORDER = 25
NUM_FILES = 250000
DATA_TYPE = 'val'
DIR_PATH = '/work/luduslab/sts_3sat/sols_by_confs/cnf_data/'

for i in range(NUM_FILES):
    file_path = DIR_PATH + f'{DATA_TYPE}/{DATA_TYPE}_no{i}.cnf'

    sts = s3s.generateSteinerTripleSystem(v=ORDER)
    s3s.sts_to_sat_file(sts=sts, order=ORDER, file_path=file_path)
    seed = ss.generate_random_seed(order=ORDER)
    ss.packaged_scrambler(base_file_path=file_path, seed=seed, write_file_path=file_path)


## Training

In [8]:
import sys 
import os
sys.path.append(os.path.abspath('..'))

from utils import sts3sat as s3s
from utils import sat_scrambler as ss

ORDER = 25
NUM_FILES = 5000000
DATA_TYPE = 'train'
DIR_PATH = '/work/luduslab/sts_3sat/sols_by_confs/cnf_data/'

for i in range(2791340):
    file_path = DIR_PATH + f'{DATA_TYPE}/{DATA_TYPE}_no{i}.cnf'

    sts = s3s.generateSteinerTripleSystem(v=ORDER)
    s3s.sts_to_sat_file(sts=sts, order=ORDER, file_path=file_path)
    seed = ss.generate_random_seed(order=ORDER)
    ss.packaged_scrambler(base_file_path=file_path, seed=seed, write_file_path=file_path)
    if i % 100000 == 0:
        print(f'{i} / {NUM_FILES} complete')


0 / 5000000 complete
100000 / 5000000 complete
200000 / 5000000 complete
300000 / 5000000 complete
400000 / 5000000 complete
500000 / 5000000 complete
600000 / 5000000 complete
700000 / 5000000 complete
800000 / 5000000 complete
900000 / 5000000 complete
1000000 / 5000000 complete
1100000 / 5000000 complete
1200000 / 5000000 complete
1300000 / 5000000 complete
1400000 / 5000000 complete
1500000 / 5000000 complete
1600000 / 5000000 complete
1700000 / 5000000 complete
1800000 / 5000000 complete
1900000 / 5000000 complete
2000000 / 5000000 complete
2100000 / 5000000 complete
2200000 / 5000000 complete
2300000 / 5000000 complete
2400000 / 5000000 complete
2500000 / 5000000 complete
2600000 / 5000000 complete
2700000 / 5000000 complete


# Run Ganak and solution counters over all cnf files to gather data



In [2]:
import sys 
import os
sys.path.append(os.path.abspath('..'))

from utils import sts_properties_from_3sat as s3sp
from utils import new_ganak_handler

GANAK_PATH = '/work/luduslab/sts_3sat/ganak/ganak'

def inner_task(args):
    job_id, data_type, num_files, dir_path, threads = args

    # lists for number of solutions for later dataframe
    num_sols = []
    # num_neg_lits = []
    pasch_counts = []
    mitre_counts = []
    fano_line_counts = []
    grid_counts = []
    prism_counts = []
    hexagon_counts = []
    crown_counts = []

    for i in range(int(num_files/threads)):
        # grab file path for each 3-SAT instance
        file_index = i + int(job_id * num_files / threads)
        sat_file_path = dir_path + f'{data_type}/{data_type}_no{file_index}.cnf'
        
        # run ganak and save results
        ganak_results = new_ganak_handler.run_ganak_parsed(cnf_path=sat_file_path,ganak_path=GANAK_PATH)
        num_sols.append(ganak_results.model_count_exact)

        # get num negative lits count
        # num_neg_lits.append(s3sp.negated_lit_count(file_path=sat_file_path))

        # lookuptable representation of literals with consideration of literals being flipped
        lookup_table = s3sp.sts3sat_lookup_table_from_file(file_path=sat_file_path)

        # count all configuration types
        pasch_counts.append(s3sp.count_pasch_configurations_lt(lookup_table))
        mitre_counts.append(s3sp.count_mitre_configurations(lookup_table))
        fano_line_counts.append(s3sp.count_fano_line_configurations(lookup_table))
        grid_counts.append(s3sp.count_grid_configurations(lookup_table))
        prism_counts.append(s3sp.count_prism_configurations(lookup_table))
        hexagon_counts.append(s3sp.count_hexagon_configurations(lookup_table))
        crown_counts.append(s3sp.count_crown_configurations(lookup_table))


    result_data = {
        f'sol_count': num_sols,
        # f'neg_lits_count': num_neg_lits,
        f'pasch_count': pasch_counts,
        f'mitre_count': mitre_counts,
        f'fano_line_count': fano_line_counts,
        f'grid_count': grid_counts,
        f'prism_count': prism_counts,
        f'hexagon_count': hexagon_counts,
        f'crown_count': crown_counts,
    }
    return result_data





## Test Data

In [ ]:
import pandas as pd
from concurrent.futures import ProcessPoolExecutor, as_completed

NUM_FILES = 100000
DATA_TYPE = 'test'
DIR_PATH = '/work/luduslab/sts_3sat/sols_by_confs/cnf_data/'
THREADS = 20


completed_workers = 0
results_dict = {}

with ProcessPoolExecutor(max_workers=THREADS) as executor:
    futures = [executor.submit(inner_task, (job_id, DATA_TYPE, NUM_FILES, DIR_PATH, THREADS)) for job_id in range(THREADS)]
    for future in as_completed(futures):
        result_data = future.result()

        for col_name, col_data in result_data.items():
            if col_name not in results_dict:
                results_dict[col_name] = []
            results_dict[col_name].extend(col_data)

        completed_workers += 1
        print(f'{completed_workers}/{THREADS} jobs completed')
            

results_csv_path = DIR_PATH + f'{DATA_TYPE}/{DATA_TYPE}_data.csv'
df = pd.DataFrame(results_dict)
df.to_csv(results_csv_path, index=False)
print(f'Results written to {results_csv_path}')

1/2 jobs completed
2/2 jobs completed
Results written to /work/luduslab/sts_3sat/sols_by_confs/cnf_data/test/test_data.csv


In [9]:
import pandas as pd
from concurrent.futures import ProcessPoolExecutor, as_completed

# NUM_FILES = 5000000
NUM_FILES = 100
DATA_TYPE = 'train'
DIR_PATH = '/work/luduslab/sts_3sat/sols_by_confs/cnf_data/'
THREADS = 20


completed_workers = 0
# results_dict = {}
results_csv_path = DIR_PATH + f'{DATA_TYPE}/{DATA_TYPE}_data.csv'
write_header = True

with ProcessPoolExecutor(max_workers=THREADS) as executor:
    futures = [executor.submit(inner_task, (job_id, DATA_TYPE, NUM_FILES, DIR_PATH, THREADS)) for job_id in range(THREADS)]
    for future in as_completed(futures):
        result_data = future.result()

        df_partial = pd.DataFrame(result_data)

        df_partial.to_csv(
            results_csv_path,
            mode='a',
            header=write_header, 
            index=False
        )

        write_header = False
        completed_workers += 1
        print(f'{completed_workers}/{THREADS} jobs completed')
            

# df = pd.DataFrame(results_dict)
# df.to_csv(results_csv_path, index=False)
print(f'Results written to {results_csv_path}')

1/20 jobs completed
2/20 jobs completed
3/20 jobs completed
4/20 jobs completed
5/20 jobs completed
6/20 jobs completed
7/20 jobs completed
8/20 jobs completed
9/20 jobs completed
10/20 jobs completed
11/20 jobs completed
12/20 jobs completed
13/20 jobs completed
14/20 jobs completed
15/20 jobs completed
16/20 jobs completed
17/20 jobs completed
18/20 jobs completed
19/20 jobs completed
20/20 jobs completed
Results written to /work/luduslab/sts_3sat/sols_by_confs/cnf_data/train/train_data.csv


# Visualization for checkpoint assignment

In [10]:
# pull first 1,000 rows from test/test_data.csv

df = pd.read_csv('test/test_data.csv')
df = df.head(1000)

df.describe()



,sol_count,neg_lits_count,pasch_count,mitre_count,fano_line_count,grid_count,prism_count,hexagon_count,crown_count,pasch_count_r,mitre_count_r,fano_line_count_r,grid_count_r,prism_count_r,hexagon_count_r,crown_count_r
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.0,1000.000000,1000.000000,1000.000000,1000.000000
mean,61.393000,149.446000,26.804000,55.078000,1.206000,139.043000,181.024000,53.511000,82.807000,0.05800,0.019000,0.0,0.074000,0.058000,0.005000,0.007000
std,96.740619,8.864743,5.410625,7.505081,1.180236,12.995385,14.373309,7.289789,8.785952,0.24227,0.143735,0.0,0.265696,0.238102,0.070569,0.083414
min,0.000000,119.000000,12.000000,35.000000,0.000000,97.000000,137.000000,32.000000,58.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,9.000000,144.000000,23.000000,50.000000,0.000000,130.000000,171.000000,49.000000,77.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
50%,29.000000,150.000000,27.000000,54.000000,1.000000,139.000000,181.000000,54.000000,82.500000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
75%,72.000000,156.000000,30.000000,60.000000,2.000000,147.250000,190.000000,58.000000,88.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
max,928.000000,176.000000,47.000000,81.000000,10.000000,184.000000,234.000000,77.000000,111.000000,2.00000,2.000000,0.0,2.000000,2.000000,1.000000,1.000000
